In [1]:
# Copyright 2024 Forusone(shins777@gmail.com)
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Controlled output generation
* [Controlled Generation](https://cloud.google.com/vertex-ai/generative-ai/docs/multimodal/control-generated-output)
  * You can guarantee that a model's generated output always adheres to a specific schema so that you receive consistently formatted responses. For example, you might have an established data schema that you use for other tasks. If you have the model follow the same schema, you can directly extract data from the model's output without any post-processing.

## Set configuration

### Install packages


In [2]:
%pip install --upgrade --user --quiet google-cloud-aiplatform

### Authenticate to access to GCP(Only Colab)

* Only for Colab in Google Drive
* No need to do this process if in Colab Enteprise on Vertex AI.

In [3]:
# To use markdown for output data from LLM
from IPython.display import display, Markdown

# Use OAuth to access the GCP environment.
import sys
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user()

## Lab execution

### Define constants

In [4]:
PROJECT_ID = "ai-hangsik"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}
MODEL_NAME = "gemini-1.5-flash-002" # @param {type:"string"}

### Import libraries

In [ ]:
import vertexai

from vertexai.generative_models import (
    GenerationConfig,
    GenerativeModel,
    HarmBlockThreshold,
    HarmCategory,
    GenerationResponse,
    Tool,
    Part,
    ChatSession
)

from vertexai.preview.generative_models import (
    grounding,
    # GenerativeModel
)
from vertexai.preview import (
    caching
)

### Initalize Vertex AI

In [ ]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest#initialization
vertexai.init(project=PROJECT_ID, location=LOCATION)


### Control output format with `response_mime_type`.
* Using `response_mime_type` when creating a model


In [ ]:
# Should set mime_type when model creation.
model = GenerativeModel(
    model_name = MODEL_NAME,
    generation_config={"response_mime_type": "application/json"},
)

# Set list format of json as a output format with prompt engineering.
prompt = """
    타이타닉 영화에 출연한 배우정보 알려주세요.

    Output format:
    actor = {"actor_name": str, "role": str}
    Return: list[actor]
"""

import json

response = model.generate_content(prompt)
json_response = json.loads(response.text)
print(json_response)


### Control output format using `response_mime_type` and `response_schema`

In [ ]:
model = GenerativeModel( model_name = MODEL_NAME,)

response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "actor_name": {
                "type": "STRING",
            },
            "role": {
                "type": "STRING",
            },

        },
        "required": ["actor_name","role"],
    },
}

response = model.generate_content(
    "타이타닉 영화에 출연한 배우정보 알려주세요.",

    generation_config=GenerationConfig(
        response_mime_type="application/json",
        response_schema=response_schema
    ),
)

print(response.text)

